In [ ]:
%matplotlib widget
from spectraclass.data.base import DataManager
from spectraclass.data.spatial.tile.manager import TileManager
from spectraclass.application.controller import app, SpectraclassController
from spectraclass.model.labels import LabelsManager, lm
from spectraclass.data.base import ModeDataManager
from typing import List, Union, Tuple, Optional, Dict, Callable
import cudf, os
from cuml.neighbors import NearestNeighbors
import xarray as xa

Here we configure paths on the Jupyter server.  If these paths are not specified here then the default values,
    defined in server-side config files, for the project ("demo2") and data mode ("desis"), will be used.  You can
    choose whatever project names you want, they are used to save configurations and results for ongoing investigations.

In [ ]:
dm: DataManager = DataManager.initialize( "demo2", 'aviris' )
dm.modal.cache_dir = "/adapt/nobackup/projects/ilab/cache"
dm.modal.data_dir = "/adapt/nobackup/projects/ilab/data/Aviris"
dm.modal.image_names = [ "ang20170720t004130_corr_v2p9" ]
TileManager.block_index = [1,1]
dm.proc_type = "gpu"
TileManager.block_size = 256
ModeDataManager.model_dims =   24

Here we define a set of class names and associated colors to be used in the classification process.

In [ ]:
classes = [ ('Class-1', "cyan"),
            ('Class-2', "green"),
            ('Class-3', "magenta"),
            ('Class-4', "blue")]

Here we start up the Spectraclass GUI.

In [ ]:
dm.loadCurrentProject()
lm().setLabels( classes )
controller: SpectraclassController = app()


Test cugraph

In [ ]:
nVerts = 10000

project_dataset: xa.Dataset = DataManager.instance().loadCurrentProject("spectraclass")
full_input = project_dataset["reduction"]
X = full_input.values[0:nVerts,:]
print(f"Dataset shape = {full_input.shape}, input shape = {X.shape}, ")
X_cudf = cudf.DataFrame(X)
print(f"X_cudf head = {X_cudf.head(10)}")
os.system("nvidia-smi")

# fit model
model = NearestNeighbors(n_neighbors=3)
model.fit(X)
os.system("nvidia-smi")

# get 3 nearest neighbors
distances, indices = model.kneighbors(X_cudf)
os.system("nvidia-smi")
print( indices.__class__ )

# print results
print(indices.head(10))
print(distances.head(10))

